# [Maximal Rectangle](https://leetcode.com/problems/maximal-rectangle/)

- Given
    + Binary matrix `A`
        + size MxN
        + filled with 0 or 1
- Find the maximum sub rectangle area
    + Note: rectangle must filled with all 1

#### Example 1

<img src="./img/7.jpg" width="250"/>

```
Output: 6
```



## Solution - DP O(N\*M)
- Cache the value into `left`, `right`, `height` dp
- Compute area by: (right - left) * height

```python
// A
0 0 0 1 0 0 0
0 0 1 1 1 0 0
0 1 1 1 1 1 0

// left
0 0 0 3 0 0 0
0 0 2 3 2 0 0
0 1 2 3 2 1 0

// right
7 7 7 4 7 7 7
7 7 5 4 5 7 7
7 6 5 4 5 4 7

// height
0 0 0 1 0 0 0
0 0 1 2 1 0 0
0 1 2 3 2 1 0

// Area
0 0 0 1 0 0 0
0 0 3 2 3 0 0
0 5 6 3 6 5 0
```

#### Code

```C++
class Solution {
public:
    int maximalRectangle(vector<vector<char>> &A) {
        int M = A.size();
        if(M == 0) return 0;
        int N = A[0].size();

        // Build left (left -> right)
        vector<vector<int>> left(M, vector<int>(N, 0));
        for(int i=0; i<M; ++i) {
            int cur_left = 0;
            for(int j=0; j<N; ++j) {
                if(A[i][j] == '1') {
                    left[i][j] = (i==0) ? cur_left : max(left[i-1][j], cur_left);
                }
                else {
                    left[i][j] = 0;
                    cur_left = j+1;
                }
            }
        }

        // Build right (right -> left)
        vector<vector<int>> right(M, vector<int>(N, N));
        for(int i=0; i<M; ++i) {
            int cur_right = N;
            for(int j=N-1; j>=0; --j) {
                if(A[i][j] == '1') {
                    right[i][j] = (i==0) ? cur_right : min(right[i-1][j], cur_right);
                }
                else {
                    right[i][j] = N;
                    cur_right = j;
                }
            }
        }

        // Build height
        vector<vector<int>> height(M, vector<int>(N, 0));
        for(int i=0; i<M; ++i) for(int j=0; j<N; ++j) { 
            if(A[i][j] == '1') {
                height[i][j] = (i==0) ? 1 : height[i-1][j] + 1;
            }
            else height[i][j] = 0;
        }

        // Find max area rec
        int ans = 0;
        for(int i=0; i<M; ++i) for(int j=0; j<N; ++j) {
            ans = max(ans,
                (right[i][j]-left[i][j])*height[i][j]);
        }
        return ans;
    }
};
```

## Solution Monotonic Stack - O(M\*N)
- Reduce the problem to [Monotonic-Stack_Largest_Rectangle_in_Histogram](https://leetcode.com/problems/largest-rectangle-in-histogram)
- For each i, We have a histogram H -> calculate the max rectangle area

<img src="./img/7.jpg" width="180"/>

```python
// H
i=0 [0]1 [1]0 [2]1 [3]0 [4]0
i=1 [0]2 [1]0 [2]2 [3]1 [4]1
i=2 [0]3 [1]1 [2]3 [3]2 [4]2
i=3 [0]4 [1]0 [2]0 [3]3 [4]0
```

#### Code

```Cpp
class Solution {
public:
    int maximalRectangle(vector<vector<char>> &A) {
        int M = A.size();
        if(M == 0) return 0;
        int N = A[0].size();

        // Create height
        vector<int> H(N, 0);
        H.push_back(0);

        int ans = 0;
        for(int i=0; i<M; ++i) {
            vector<int> stck;
            for(int r=0; r<H.size(); ++r) {
                // Update value for H
                if(r < N) {
                    if(A[i][r] == '1') H[r] += 1;
                    else H[r] = 0;
                }

                // Pop stack
                while(!stck.empty() && H[stck.back()] >= H[r]) {
                    int j = stck.back();
                    stck.pop_back();

                    int l = stck.empty() ? -1 : stck.back();
                    int area = (r-l-1) * H[j];

                    // Relax ans
                    ans = max(ans, area);
                }

                // Push stack
                stck.push_back(r);
            }
        }
        return ans;
    }
};
```